In [ ]:
# !pip install google-cloud-bigquery

In [1]:
import pandas as pd
import re
import os
import datetime
import numpy as np
from google.cloud import bigquery
import uuid
import decimal

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\hu-cloud-data-storage-e81319c069ef.json"

In [3]:
client = bigquery.Client()

In [4]:
dataset_id = 'Stockcards'
table_id = 'Majid Stock Card 2'

In [5]:
table_ref = f'{client.project}.{dataset_id}.{table_id}'

In [6]:
table = client.get_table(table_ref)
schema = table.schema

In [7]:
schema

[SchemaField('Date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('Status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('ItemNumberMajid', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Barcode', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('ItemNumberHU', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('ItemDescription', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Department', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Brand', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Customer', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('CustomerCode', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('InstockCustomer', 'NUMERIC', 'NULLABLE', None, None, (), None),
 SchemaField('MainWarehouse', 'NUMERIC', 'NULLABLE', None, None, (), None),
 SchemaField('ProductionWarehouse', 'NUMERIC', 'NULLABLE', None, None, (), None),
 SchemaField('RetailWarehouse', 'NUMERIC', 'NULLABLE

In [12]:
path = "C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Majid Stock Cards"

In [13]:
cols = ['STATUS','ItemNo_Majid','Bar Code',	'Item No.',	'Item Description',
'Department', 'Brand', 'Healthy U Acc Name',	'Healthy U Acc Code',	'Instock_Majid', 'MAIN WAREHOUSE', 'PRODUCTION WAREHOUSE',	
'RETAIL WAREHOUSE', 'PACKAGING MATERIALS WAREHOUSE',	'RECEIVING WAREHOUSE']

In [14]:
file_names=[]
for i in os.listdir(path):
    file_names.append(i.replace('.xlsx', ""))

In [15]:
file_names

['5_7_2024']

In [16]:
fin_table = pd.DataFrame()
for i in file_names:
    print(i)
    day = int(re.findall('\d+', i)[0])
    month = int(re.findall('\d+', i)[1])
    year = int(re.findall('\d+', i)[2])
    
    file_name="C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Majid Stock Cards\\{x}_{y}_{z}.xlsx"
    temp_table=pd.read_excel(file_name.format(x=day, y=month, z=year))
    temp_table2 = temp_table.copy()
    temp_table2 = temp_table2[cols]
    temp_table2.insert(0, 'Date', datetime.date(year, month,day))
    fin_table = pd.concat([fin_table, temp_table2])
    

5_7_2024


In [17]:
fin_table= fin_table.set_axis(range(len(fin_table)))

In [18]:
fin_table['Bar Code'] = fin_table['Bar Code'].apply(lambda x: '{:.0f}'.format(x))

In [19]:
# fin_table2['Bar Code'].replace([np.inf, -np.inf], np.nan)

# fin_table2['Bar Code'] = fin_table2['Bar Code'].fillna('XXX')

In [20]:
fin_table2=fin_table.copy()

In [21]:
fin_table2.drop(fin_table2[fin_table2['Bar Code']=='nan'].index, inplace=True)

In [22]:
fin_table2['Item No.'] = fin_table2['Item No.'].astype(int)

In [23]:
fin_table2['Instock_Majid']=fin_table2['Instock_Majid'].fillna(0)

In [24]:
fin_table2.columns = ['Date', 'Status', 'ItemNumberMajid', 'Barcode', 'ItemNumberHU', 'ItemDescription', 'Department', 'Brand',
                      'Customer', 'CustomerCode', 'InstockCustomer', 'MainWarehouse', 'ProductionWarehouse', 'RetailWarehouse',
                      'PackagingWarehouse', 'ReceivingWarehouse']

In [25]:
fin_table2.loc[:, 'InstockCustomer': 'ReceivingWarehouse'] = fin_table2.loc[:, 'InstockCustomer': 'ReceivingWarehouse'].fillna(0)

In [26]:
fin_table2['ItemNumberMajid']=fin_table2['ItemNumberMajid'].astype(str)
fin_table2['ItemNumberHU']=fin_table2['ItemNumberHU'].astype(str)

In [27]:
fin_table2.loc[:, 'InstockCustomer':]= fin_table2.loc[:, 'InstockCustomer':].astype(str).map(decimal.Decimal)

In [28]:
fin_table2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44252 entries, 0 to 63410
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 44252 non-null  object
 1   Status               44252 non-null  object
 2   ItemNumberMajid      44252 non-null  object
 3   Barcode              44252 non-null  object
 4   ItemNumberHU         44252 non-null  object
 5   ItemDescription      44252 non-null  object
 6   Department           44252 non-null  object
 7   Brand                44252 non-null  object
 8   Customer             44252 non-null  object
 9   CustomerCode         44252 non-null  object
 10  InstockCustomer      44252 non-null  object
 11  MainWarehouse        44252 non-null  object
 12  ProductionWarehouse  44252 non-null  object
 13  RetailWarehouse      44252 non-null  object
 14  PackagingWarehouse   44252 non-null  object
 15  ReceivingWarehouse   44252 non-null  object
dtypes: object

### LOADING

In [29]:
fin_table3=fin_table2.drop_duplicates()

In [30]:
fin_table3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43401 entries, 0 to 63410
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 43401 non-null  object
 1   Status               43401 non-null  object
 2   ItemNumberMajid      43401 non-null  object
 3   Barcode              43401 non-null  object
 4   ItemNumberHU         43401 non-null  object
 5   ItemDescription      43401 non-null  object
 6   Department           43401 non-null  object
 7   Brand                43401 non-null  object
 8   Customer             43401 non-null  object
 9   CustomerCode         43401 non-null  object
 10  InstockCustomer      43401 non-null  object
 11  MainWarehouse        43401 non-null  object
 12  ProductionWarehouse  43401 non-null  object
 13  RetailWarehouse      43401 non-null  object
 14  PackagingWarehouse   43401 non-null  object
 15  ReceivingWarehouse   43401 non-null  object
dtypes: object

In [31]:
job = client.load_table_from_dataframe(fin_table3, table_ref, job_config=bigquery.LoadJobConfig(schema=schema))
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Loaded 43401 rows into Stockcards:Majid Stock Card 2.


### ACTIVE ITEMS

In [ ]:
active_items = pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Active Item List 24_05.xlsx")

In [ ]:
positive_inventory = pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Positive Inventory\\Positive Inventory (Retail) 24.06.2024.xlsx")

In [ ]:
active_shortened = active_items [['Item No.', 'Item Description','Bar Code', 'Brand', 'Department', 'Last Purchase Price', 'Retail Price Inc VAT', 'Retail Price Ex. VAT']].copy()

In [ ]:
positive_shortened = positive_inventory [['Item No.', 'Barcode', 'Item Description', 'Department', 'Brand', 'RETAIL WAREHOUSE', 'MAIN WAREHOUSE', 'RECEIVING WAREHOUSE']].copy()

In [ ]:
active_shortened.loc[:, 'Last Purchase Price':]= active_shortened.loc[:, 'Last Purchase Price':].astype(str).map(decimal.Decimal)

In [ ]:
table_id_activeitem = 'ActiveItemList'

In [ ]:
table_ref_activeitem = f'{client.project}.{dataset_id}.{table_id_activeitem}'

In [ ]:
table_activeitem = client.get_table(table_ref_activeitem)
schema_activeitem = table_activeitem.schema

In [ ]:
schema_activeitem

In [ ]:
newcolsactiveitem = []
for i in range(len (schema_activeitem)):
    newcolsactiveitem.append(str(schema_activeitem[i]).replace("(","").replace("SchemaField", "").split(",")[0].replace("'", ""))

In [ ]:
active_shortened.columns=newcolsactiveitem

In [ ]:
job_activeitem = client.load_table_from_dataframe(active_shortened, table_ref_activeitem, job_config=bigquery.LoadJobConfig(schema=schema_activeitem))
job_activeitem.result()

print(f"Loaded {job_activeitem.output_rows} rows into {dataset_id}:{table_id_activeitem}.")